In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Leo la tabla de películas
movies = pd.read_csv('data/movies/movies.csv')

In [3]:
# Leo cuántos tags diferentes tengo
tags = []
for indx in movies.index:
    genres = movies.at[indx, 'genres']
    genres_list = genres.split('|')
    tags.extend(genres_list)
    tags = list(set(tags))

In [4]:
print (f'All possible tags are: {", ".join(tags)}')

All possible tags are: Animation, Mystery, Action, Documentary, Fantasy, Adventure, Crime, Western, Drama, (no genres listed), Romance, Children, Sci-Fi, Comedy, Musical, War, Horror, Film-Noir, Thriller, IMAX


In [5]:
# Creo una tabla con (0,1) para cada película/categoría
tagged_data = []
for indx in movies.index:
    genres = movies.at[indx, 'genres']
    genres_list = genres.split('|')
    movie_tags_list = [1 if t in genres_list else 0 for t in tags]
    tagged_data.append(movie_tags_list)

In [6]:
# Ahora, guardo esa información de tags en un DataFrame de pandas
data = pd.DataFrame(tagged_data, columns=tags)

In [7]:
# para que vean cómo se ve
data[:2]

,Animation,Mystery,Action,Documentary,Fantasy,Adventure,Crime,Western,Drama,(no genres listed),Romance,Children,Sci-Fi,Comedy,Musical,War,Horror,Film-Noir,Thriller,IMAX
0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# KMeans

In [8]:
# En este contexto de aprendizaje no supervisado, no hay target values: es decir, nuay y
X = tagged_data
np.shape(X)

(9742, 20)

In [9]:
# Utilizaré KMeans para armar un sistemita de recomendaciones.
from sklearn.cluster import KMeans

In [10]:
# haré sólo 10 clusters.
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=82).fit(X)

In [11]:
pelis, _ = movies.shape

### Sistema de recomendación

In [15]:
#Selecciono una pelicula equis
peli = np.random.randint(pelis)
peli = 0 # Uncomment this line to set peli = 0, selecting Toy Story
#Obtengo sus generos
peli_name = movies.at[peli,'title']
peli_genres = movies.at[peli, 'genres']
#Separo los tags
peli_genres_list = peli_genres.split('|')
# peli_genres_list = ['Action','Western','Fantasy'] # Uncomment these linen for La película rara de Flor
# peli_name = 'La pelicula rara de Flor'  # Uncomment these linen for La película rara de Flor
peli_tags_list = [1 if t in peli_genres_list else 0 for t in tags]
peli_cluster = kmeans.predict([peli_tags_list])

In [16]:
peli_name, peli_cluster[0]

('Toy Story (1995)', 2)

In [19]:
# REPORTING:
recomendeishon = [i for i,cl in enumerate(kmeans.labels_) if (cl == peli_cluster and i != peli)]
count = len(recomendeishon)
random.shuffle(recomendeishon)
print (f'If you like the movie "{peli_name}", you may also like:')
p_recomendeishon = movies.loc[recomendeishon,'title']   # Acá, movies.at no funciona porque usamos lista de indices
[print(f'\t- {TITLE}') for TITLE in p_recomendeishon[:10]];

If you like the movie "Toy Story (1995)", you may also like:
	- Town Called Panic, A (Panique au village) (2009)
	- My Science Project (1985)
	- Mystery of the Third Planet, The (Tayna tretey planety) (1981)
	- History of Future Folk, The (2012)
	- Madagascar (2005)
	- Pinocchio (1940)
	- Valiant (2005)
	- Paul (2011)
	- Planes: Fire & Rescue (2014)
	- Three Musketeers, The (2011)


In [ ]:
# método ilustrativo. muestra paso a paso qué se haría para un sistema de recomendaciones
# no es óptimo porque estamos recalculando agrupamientos que ya teníamos listos en labels_.
if False:
    # REPORTING:
    print (f'If you like the movie "{peli_name}", you may also like:')
    recomendeishon = []
    count = 0
    for indx in movies.index:
        p_genres = movies.at[indx, 'genres']
        p_genres_list = p_genres.split('|')
        p_tags_list = [1 if t in p_genres_list else 0 for t in tags]
        p_cluster = kmeans.predict([p_tags_list])
        if p_cluster == peli_cluster and peli != indx:
            count += 1
            recomendeishon.append(movies.at[indx, 'title'])
            #if count >= 10:
            #    break
    # En lugar de este random, se mezclaría con raitings 
    random.shuffle(recomendeishon)
    [print(f'\t- {TITLE}') for TITLE in recomendeishon[:10]];